This notebook was for me [Tim] to test different evaluation techniques. I tried to apply different metrics to the point-to-point distance between two clouds which worked reasonabbly well. I however failed tothe the point to mesh distance since it was to computationally expensive. Lining up the stitched point cloud with the original model also proved to be difficult.

## Setup

In [1]:
import open3d as o3d
import numpy as np
import copy
import trimesh
import threading
import time
from multiprocessing.pool import ThreadPool

%run "../Notebooks/ClusteringPlottingMeshing.ipynb"
%run "../Notebooks/Stitching_functions.ipynb"

Clustering, Plotting and Meshing loaded at


In [4]:
#load original mesh, scale it anf convert it to point cloud
original_mesh = o3d.io.read_triangle_mesh("../data/dental.stl")
original_mesh.scale(1000, [0,0,0])
original_mesh.compute_vertex_normals()

original_pcd = o3d.geometry.PointCloud()
original_pcd.points = original_mesh.vertices
original_pcd.estimate_normals()
original_pcd.paint_uniform_color([0.2,0.2,0.8])

stitched_pcd = o3d.io.read_point_cloud("../data/st_cropped_2.ply")
stitched_pcd.estimate_normals()
stitched_pcd.paint_uniform_color([0.8,0.2,0.2])


PointCloud with 299291 points.

In [5]:
#make copies to not break the original
org_mesh_copy = copy.deepcopy(original_mesh)
org_pcd_copy = copy.deepcopy(original_pcd)
st_pcd_copy = copy.deepcopy(stitched_pcd)


transformation_setup = np.linalg.inv(np.array([
     [ -0.42936695,   0.89689573,  -0.10593428,  18.08900141],
     [ -0.73091052,  -0.4139897 ,  -0.54257012,  -4.68199364],
     [ -0.53048452,  -0.15553319,   0.83330402, -14.77113242],
     [  0.        ,   0.        ,   0.        ,   1.        ]]))

st_pcd_copy.transform(transformation_setup)

PointCloud with 299291 points.

In [ ]:
o3d.visualization.draw_geometries([org_mesh_copy,st_pcd_copy], mesh_show_back_face=True)

## Point to Point metrics

In [ ]:
dists = np.asarray(org_pcd_copy.compute_point_cloud_distance(st_pcd_copy))
print("Hausdorff:", max(dists))
print("Median:", np.median(dists))
print("Mean:", np.mean(dists))
print("Mean Squared Error:", np.mean(dists*dists))

## Point to Mesh Metrics

In [23]:
#crop model to reduce processing time
boundingbox = o3d.geometry.OrientedBoundingBox.create_from_points(target.points)
cropped_mesh = copy.deepcopy(model_mesh).crop(boundingbox) 

o3d.visualization.draw_geometries([cropped_mesh,target,boundingbox], mesh_show_back_face=True)

In [24]:
#Scale down point cloud size because the algorithm runs in O(mesh_size*cloud_size) which makes it computationally very expensive 
voxel_size = 1.0
st_pcd_down = target.voxel_down_sample(voxel_size)

o3d.visualization.draw_geometries([st_pcd_down])

In [26]:
dists2 = abs(trimesh.proximity.signed_distance(convert(cropped_mesh), np.asarray(st_pcd_down.points)))

In [27]:
print("Hausdorff:", max(dists2))
print("Median:", np.median(dists2))
print("Mean:", np.mean(dists2))
print("Mean Squared Error:", np.mean(dists2*dists2))

Hausdorff: 1.1257412077449076
Median: 0.20450815462755279
Mean: 0.2682834633637908
Mean Squared Error: 0.1249605028407107


Final note: Both approaches give to very different results. I cannot say why and I don't know which is correct.